In [21]:
import pandas as pd
import numpy as np
import os
from sqlalchemy import create_engine
from sqlalchemy.exc import ProgrammingError
import warnings
from animal_munging import *

In [2]:
intakes_file_path = os.path.join("center_data", "Austin_Animal_Center_Intakes.csv")

dog_intakes_df = pd.read_csv(intakes_file_path)\
    .rename(columns = lambda df: df.lower().replace(" ", "_"))\
    .drop("monthyear", axis=1)

dog_intakes_df = dog_intakes_df[dog_intakes_df["animal_type"] == 'Dog']\
    .rename(columns = {
    "intake_type":"type",
    "sex_upon_intake":"sex",
    "age_upon_intake":"age"
    })\
    .assign(is_intake=True)\
    .assign(datetime = lambda df: pd.to_datetime(df["datetime"]))\
    .reset_index(drop=True)

# dogs_intakes_df.to_csv('dog_intakes.csv')
dog_intakes_df.head(5)

,animal_id,name,datetime,found_location,type,intake_condition,animal_type,sex,age,breed,color,is_intake
0,A786884,*Brock,2019-01-03 16:19:00,2501 Magin Meadow Dr in Austin (TX),Stray,Normal,Dog,Neutered Male,2 years,Beagle Mix,Tricolor,True
1,A706918,Belle,2015-07-05 12:59:00,9409 Bluegrass Dr in Austin (TX),Stray,Normal,Dog,Spayed Female,8 years,English Springer Spaniel,White/Liver,True
2,A724273,Runster,2016-04-14 18:43:00,2818 Palomino Trail in Austin (TX),Stray,Normal,Dog,Intact Male,11 months,Basenji Mix,Sable/White,True
3,A682524,Rio,2014-06-29 10:38:00,800 Grove Blvd in Austin (TX),Stray,Normal,Dog,Neutered Male,4 years,Doberman Pinsch/Australian Cattle Dog,Tan/Gray,True
4,A743852,Odin,2017-02-18 12:46:00,Austin (TX),Owner Surrender,Normal,Dog,Neutered Male,2 years,Labrador Retriever Mix,Chocolate,True


In [3]:
outcomes_file_path = os.path.join("center_data", "Austin_Animal_Center_Outcomes.csv")

dog_outcomes_df = pd.read_csv(outcomes_file_path)\
    .rename(columns = lambda df: df.lower().replace(" ", "_"))\
    .drop("monthyear", axis=1)

dog_outcomes_df = dog_outcomes_df[
    (dog_outcomes_df["animal_type"] == 'Dog') & (dog_outcomes_df['outcome_type'] == 'Adoption')]\
    .rename(columns = {
        "outcome_type":"type",
        "sex_upon_outcome":"sex",
        "age_upon_outcome":"age"
        })\
    .assign(is_intake=False)\
    .assign(datetime = lambda df: pd.to_datetime(df["datetime"]))\
    .reset_index(drop=True)    

# dogs_intakes_df.to_csv('dog_intakes.csv')
dog_outcomes_df.head(5)

,animal_id,name,datetime,date_of_birth,type,outcome_subtype,animal_type,sex,age,breed,color,is_intake
0,A720371,Moose,2016-02-13 17:59:00,10/08/2015,Adoption,NaN,Dog,Neutered Male,4 months,Anatol Shepherd/Labrador Retriever,Buff,False
1,A725530,King,2016-06-09 14:50:00,10/30/2015,Adoption,NaN,Dog,Neutered Male,7 months,Pit Bull Mix,Brown Brindle/White,False
2,A738919,Milo,2016-11-27 17:35:00,09/23/2016,Adoption,NaN,Dog,Neutered Male,2 months,Black Mouth Cur Mix,Brown/Black,False
3,A665383,Indica,2013-10-30 16:23:00,06/19/2013,Adoption,NaN,Dog,Spayed Female,4 months,German Shepherd Mix,Black/Tan,False
4,A689228,Louie,2014-12-05 15:55:00,06/01/2014,Adoption,NaN,Dog,Neutered Male,6 months,Dachshund/Chihuahua Shorthair,Sable/Tan,False


In [4]:
dog_concat_df = pd\
    .concat([dog_intakes_df, dog_outcomes_df], sort=False)\
    .sort_values(by=["animal_id", "datetime"])\
     [lambda df: df.duplicated(["animal_id"],keep=False)]\
    .fillna("N/A")

dog_concat_df.head()

,animal_id,name,datetime,found_location,type,intake_condition,animal_type,sex,age,breed,color,is_intake,date_of_birth,outcome_subtype
39206,A006100,Scamp,2014-03-07 14:26:00,8700 Research in Austin (TX),Public Assist,Normal,Dog,Neutered Male,6 years,Spinone Italiano Mix,Yellow/White,True,N/A,N/A
1859,A006100,Scamp,2014-12-19 10:21:00,8700 Research Blvd in Austin (TX),Public Assist,Normal,Dog,Neutered Male,7 years,Spinone Italiano Mix,Yellow/White,True,N/A,N/A
8663,A006100,Scamp,2017-12-07 14:07:00,Colony Creek And Hunters Trace in Austin (TX),Stray,Normal,Dog,Neutered Male,10 years,Spinone Italiano Mix,Yellow/White,True,N/A,N/A
45812,A200922,Carlos,2013-10-03 15:47:00,Austin (TX),Owner Surrender,Normal,Dog,Neutered Male,16 years,Dachshund Mix,Black/Tan,True,N/A,N/A
5218,A200922,Carlos,2013-11-22 09:44:00,N/A,Adoption,N/A,Dog,Neutered Male,16 years,Dachshund Mix,Black/Tan,False,10/03/1997,Foster


In [5]:
intakes_agg = dog_concat_df[dog_concat_df["is_intake"]].groupby("animal_id")["animal_id"].count()
outcomes_agg = dog_concat_df[~dog_concat_df["is_intake"]].groupby("animal_id")["animal_id"].count()

In [6]:
validation_agg = pd.merge(
    intakes_agg.to_frame().rename(columns = {"animal_id":"intake_count"}),
    outcomes_agg.to_frame().rename(columns = {"animal_id":"outcome_count"}),
    how="outer",
    left_index=True,
    right_index=True
).dropna().reset_index()

validation_agg["valid"] = np.vectorize(qualified)(validation_agg["intake_count"], validation_agg["outcome_count"])
matched_stays_df = validation_agg[validation_agg["valid"]]["animal_id"]

In [7]:
working_df = dog_concat_df.loc[dog_concat_df["animal_id"].isin(matched_stays_df.values)].reset_index(drop=True)
working_df.head(5)

,animal_id,name,datetime,found_location,type,intake_condition,animal_type,sex,age,breed,color,is_intake,date_of_birth,outcome_subtype
0,A200922,Carlos,2013-10-03 15:47:00,Austin (TX),Owner Surrender,Normal,Dog,Neutered Male,16 years,Dachshund Mix,Black/Tan,True,N/A,N/A
1,A200922,Carlos,2013-11-22 09:44:00,N/A,Adoption,N/A,Dog,Neutered Male,16 years,Dachshund Mix,Black/Tan,False,10/03/1997,Foster
2,A210457,Caleb,2016-09-28 12:05:00,4424 S Mopac #412 in Austin (TX),Public Assist,Aged,Dog,Neutered Male,17 years,Chihuahua Shorthair,Tan/Black,True,N/A,N/A
3,A210457,Caleb,2016-10-07 12:34:00,N/A,Adoption,N/A,Dog,Neutered Male,17 years,Chihuahua Shorthair,Tan/Black,False,06/01/1999,Foster
4,A226069,Cedar,2015-10-06 12:29:00,Wheless Ln And Berkman Dr in Austin (TX),Stray,Normal,Dog,Neutered Male,15 years,Labrador Retriever/Beagle,Sable/White,True,N/A,N/A


In [8]:
# Colors
   
total_colors = total_color_set(working_df["color"])

color_conversion_dict = {
    "Black":[color for color in total_colors if color.startswith("Black")],
    "Brown":[color for color in total_colors if color.startswith("Brown")],
    "Blue":[color for color in total_colors if color.startswith("Blue")],
    "Red":[color for color in total_colors if color.startswith("Red")],
    "Yellow":[color for color in total_colors if color.startswith("Yellow")],
    "Gray":["Gray", "Silver"],
    "Tan":["Tan", "Buff", "Orange", "Gold", "Fawn", "Apricot"],
    "White":["White", "Cream"],
    "Tricolor":["Tricolor", "Calico", "Sable"],
    "Pink": "Pink"
}
color_conversion_dict["Red"].append("Ruddy")
color_conversion_dict["Brown"].extend(["Agouti", "Chocolate", "Liver", "Liver Tick"])

In [9]:
working_df["color_primary"] = working_df["color"]\
    .apply(split_colors)\
    .apply(lambda color: convert_color(color, color_conversion_dict))

working_df["color_secondary"] = working_df["color"]\
    .apply(lambda color: split_colors(color, primary=False))\
    .apply(lambda color: convert_color(color, color_conversion_dict))

working_df = working_df.drop("color", axis=1)
# Do not run this cell again or else drop won't work

In [10]:
# Breeds

working_df.loc[working_df['breed'].str.contains('Labrador Retriever/'), 'breed'] = 'Labrador Retriever Mix'
working_df.loc[working_df['breed'].str.contains('Chihuahua Shorthair/'), 'breed'] = 'Chihuahua Shorthair Mix'
working_df.loc[working_df['breed'].str.contains('Pit Bull/'), 'breed'] = 'Pit Bull Mix'
working_df.loc[working_df['breed'].str.contains('German Shepherd/'), 'breed'] = 'German Shepherd Mix'
working_df.loc[working_df['breed'].str.contains('Australian Cattle Dog/'), 'breed'] = 'Australian Cattle Dog Mix'
working_df.loc[working_df['breed'].str.contains('Dachshund/'), 'breed'] = 'Dachshund Mix'
working_df.loc[working_df['breed'].str.contains('Border Collie/'), 'breed'] = 'Border Collie Mix'
working_df.loc[working_df['breed'].str.contains('Boxer/'), 'breed'] = 'Boxer Mix'
working_df.loc[working_df['breed'].str.contains('Catahoula/'), 'breed'] = 'Catahoula Mix'
working_df.loc[working_df['breed'].str.contains('Miniature Poodle/'), 'breed'] = 'Miniature Poodle Mix'
working_df.loc[working_df['breed'].str.contains('Australian Shepherd/'), 'breed'] = 'Australian Shepherd Mix'
working_df.loc[working_df['breed'].str.contains('Pointer/'), 'breed'] = 'Pointer Mix'
working_df.loc[working_df['breed'].str.contains('Jack Russell Terrier/'), 'breed'] = 'Jack Russell Terrier Mix'
working_df.loc[working_df['breed'].str.contains('Beagle/'), 'breed'] = 'Beagle Mix'
working_df.loc[working_df['breed'].str.contains('Yorkshire Terrier/'), 'breed'] = 'Yorkshire Terrier Mix'
working_df.loc[working_df['breed'].str.contains('Rat Terrier/'), 'breed'] = 'Rat Terrier Mix'
working_df.loc[working_df['breed'].str.contains('Cairn Terrier/'), 'breed'] = 'Cairn Terrier Mix'
working_df.loc[working_df['breed'].str.contains('Staffordshire/'), 'breed'] = 'Staffordshire Mix'
working_df.loc[working_df['breed'].str.contains('Great Pyrenees/'), 'breed'] = 'Great Pyrenees Mix'
working_df.loc[working_df['breed'].str.contains('Miniature Schnauzer/'), 'breed'] = 'Miniature Schnauzer Mix'
working_df.loc[working_df['breed'].str.contains('Siberian Husky/'), 'breed'] = 'Siberian Husky Mix'
working_df.loc[working_df['breed'].str.contains('Chihuahua Longhair/'), 'breed'] = 'Chihuahua Longhair Mix'
working_df.loc[working_df['breed'].str.contains('Anatol Shepherd/'), 'breed'] = 'Anatol Shepherd Mix'
working_df.loc[working_df['breed'].str.contains('Plott Hound/'), 'breed'] = 'Plott Hound Mix'
working_df.loc[working_df['breed'].str.contains('Black Mouth Cur/'), 'breed'] = 'Black Mouth Cur Mix'
working_df.loc[working_df['breed'].str.contains('Rottweiler/'), 'breed'] = 'Rottweiler Mix'
working_df.loc[working_df['breed'].str.contains('Australian Kelpie/'), 'breed'] = 'Australian Kelpie Mix'
working_df.loc[working_df['breed'].str.contains('American Bulldog/'), 'breed'] = 'American Bulldog Mix'
working_df.loc[working_df['breed'].str.contains('Miniature Pinscher/'), 'breed'] = 'Miniature Pinscher Mix'
working_df.loc[working_df['breed'].str.contains('Border Terrier/'), 'breed'] = 'Border Terrier Mix'
working_df.loc[working_df['breed'].str.contains('American Pit Bull Terrier/'), 'breed'] = 'American Pit Bull Terrier Mix'
working_df.loc[working_df['breed'].str.contains('Cardigan Welsh Corgi/'), 'breed'] = 'Cardigan Welsh Corgi Mix'
working_df.loc[working_df['breed'].str.contains('Golden Retriever/'), 'breed'] = 'Golden Retriever Mix'
working_df.loc[working_df['breed'].str.contains('Maltese/'), 'breed'] = 'Maltese Mix'
working_df.loc[working_df['breed'].str.contains('Blue Lacy/'), 'breed'] = 'Blue Lacy Mix'
working_df.loc[working_df['breed'].str.contains('Queensland Heeler/'), 'breed'] = 'Queensland Heeler Mix'
working_df.loc[working_df['breed'].str.contains('American Staffordshire Terrier/'), 'breed'] = 'American Staffordshire Terrier Mix'
working_df.loc[working_df['breed'].str.contains('Basset Hound/'), 'breed'] = 'Basset Hound Mix'
working_df.loc[working_df['breed'].str.contains('Manchester Terrier/'), 'breed'] = 'Manchester Terrier Mix'
working_df.loc[working_df['breed'].str.contains('Black/Tan Hound/'), 'breed'] = 'Black/Tan Hound Mix'
working_df.loc[working_df['breed'].str.contains('Collie Smooth/'), 'breed'] = 'Collie Smooth Mix'
working_df.loc[working_df['breed'].str.contains('Norfolk Terrier/'), 'breed'] = 'Norfolk Terrier Mix'
working_df.loc[working_df['breed'].str.contains('Dachshund Longhair/'), 'breed'] = 'Dachshund Longhair Mix'
working_df.loc[working_df['breed'].str.contains('Redbone Hound/'), 'breed'] = 'Redbone Hound Mix'
working_df.loc[working_df['breed'].str.contains('Dachshund Wirehair/'), 'breed'] = 'Dachshund Wirehair Mix'
working_df.loc[working_df['breed'].str.contains('Doberman Pinsch/'), 'breed'] = 'Doberman Pinsch Mix'
working_df.loc[working_df['breed'].str.contains('Flat Coat Retriever/'), 'breed'] = 'Flat Coat Retriever Mix'
working_df.loc[working_df['breed'].str.contains('Mastiff/'), 'breed'] = 'Mastiff Mix'
working_df.loc[working_df['breed'].str.contains('Shih Tzu/'), 'breed'] = 'Shih Tzu Mix'
working_df.loc[working_df['breed'].str.contains('Chow Chow/'), 'breed'] = 'Chow Chow Mix'
working_df.loc[working_df['breed'].str.contains('Carolina Dog/'), 'breed'] = 'Carolina Dog Mix'
working_df.loc[working_df['breed'].str.contains('Harrier/'), 'breed'] = 'Harrier Mix'
working_df.loc[working_df['breed'].str.contains('Toy Poodle/'), 'breed'] = 'Toy Poodle Mix'
working_df.loc[working_df['breed'].str.contains('Whippet/'), 'breed'] = 'Whippet Mix'
working_df.loc[working_df['breed'].str.contains('Chinese Sharpei/'), 'breed'] = 'Chinese Sharpei Mix'
working_df.loc[working_df['breed'].str.contains('Alaskan Husky/'), 'breed'] = 'Alaskan Husky Mix'
working_df.loc[working_df['breed'].str.contains('Pembroke Welsh Corgi/'), 'breed'] = 'Pembroke Welsh Corgi Mix'
working_df.loc[working_df['breed'].str.contains('Basenji/'), 'breed'] = 'Basenji Mix'
working_df.loc[working_df['breed'].str.contains('Rhod Ridgeback/'), 'breed'] = 'Rhod Ridgeback Mix'
working_df.loc[working_df['breed'].str.contains('Cocker Spaniel/'), 'breed'] = 'Cocker Spaniel Mix'
working_df.loc[working_df['breed'].str.contains('Great Dane/'), 'breed'] = 'Great Dane Mix'
working_df.loc[working_df['breed'].str.contains('Soft Coated Wheaten Terrier/'), 'breed'] = 'Soft Coated Wheaten Terrier Mix'
working_df.loc[working_df['breed'].str.contains('Pomeranian/'), 'breed'] = 'Pomeranian Mix'
working_df.loc[working_df['breed'].str.contains('Bruss Griffon/'), 'breed'] = 'Bruss Griffon Mix'
working_df.loc[working_df['breed'].str.contains('Pug/'), 'breed'] = 'Pug Mix'
working_df.loc[working_df['breed'].str.contains('Norwich Terrier/'), 'breed'] = 'Norwich Terrier Mix'
working_df.loc[working_df['breed'].str.contains('Belgian Malinois/'), 'breed'] = 'Belgian Malinois Mix'
working_df.loc[working_df['breed'].str.contains('Wire Hair Fox Terrier/'), 'breed'] = 'Wire Hair Fox Terrier Mix'
working_df.loc[working_df['breed'].str.contains('English Bulldog/'), 'breed'] = 'English Bulldog Mix'
working_df.loc[working_df['breed'].str.contains('Boston Terrier/'), 'breed'] = 'Boston Terrier Mix'
working_df.loc[working_df['breed'].str.contains('German Shorthair Pointer/'), 'breed'] = 'German Shorthair Pointer Mix'
working_df.loc[working_df['breed'].str.contains('Papillon/'), 'breed'] = 'Papillon Mix'
working_df.loc[working_df['breed'].str.contains('Tibetan Spaniel/'), 'breed'] = 'Tibetan Spaniel Mix'
working_df.loc[working_df['breed'].str.contains('Smooth Fox Terrier/'), 'breed'] = 'Smooth Fox Terrier Mix'
working_df.loc[working_df['breed'].str.contains('Dalmatian/'), 'breed'] = 'Dalmatian Mix'
working_df.loc[working_df['breed'].str.contains('Pbgv/'), 'breed'] = 'Pbgv Mix'
working_df.loc[working_df['breed'].str.contains('Bull Terrier/'), 'breed'] = 'Bull Terrier Mix'
working_df.loc[working_df['breed'].str.contains('Vizsla/'), 'breed'] = 'Vizsla Mix'
working_df.loc[working_df['breed'].str.contains('Standard Schnauzer/'), 'breed'] = 'Standard Schnauzer Mix'
working_df.loc[working_df['breed'].str.contains('Weimaraner/'), 'breed'] = 'Weimaraner Mix'
working_df.loc[working_df['breed'].str.contains('Scottish Terrier/'), 'breed'] = 'Scottish Terrier Mix'
working_df.loc[working_df['breed'].str.contains('Dutch Shepherd/'), 'breed'] = 'Dutch Shepherd Mix'
working_df.loc[working_df['breed'].str.contains('Pekingese/'), 'breed'] = 'Pekingese Mix'
working_df.loc[working_df['breed'].str.contains('Akita/'), 'breed'] = 'Akita Mix'
working_df.loc[working_df['breed'].str.contains('Pharaoh Hound/'), 'breed'] = 'Pharaoh Hound Mix'
working_df.loc[working_df['breed'].str.contains('Collie Rough/'), 'breed'] = 'Collie Rough Mix'
working_df.loc[working_df['breed'].str.contains('Bichon Frise/'), 'breed'] = 'Bichon Frise Mix'
working_df.loc[working_df['breed'].str.contains('English Coonhound/'), 'breed'] = 'English Coonhound Mix'
working_df.loc[working_df['breed'].str.contains('Italian Greyhound/'), 'breed'] = 'Italian Greyhound Mix'
working_df.loc[working_df['breed'].str.contains('Shiba Inu/'), 'breed'] = 'Shiba Inu Mix'
working_df.loc[working_df['breed'].str.contains('English Pointer/'), 'breed'] = 'English Pointer Mix'
working_df.loc[working_df['breed'].str.contains('Greyhound/'), 'breed'] = 'Greyhound Mix'
working_df.loc[working_df['breed'].str.contains('Lhasa Apso/'), 'breed'] = 'Lhasa Apso Mix'
working_df.loc[working_df['breed'].str.contains('Havanese/'), 'breed'] = 'Havanese Mix'
working_df.loc[working_df['breed'].str.contains('Shetland Sheepdog/'), 'breed'] = 'Shetland Sheepdog Mix'
working_df.loc[working_df['breed'].str.contains('West Highland/'), 'breed'] = 'West Highland Mix'
working_df.loc[working_df['breed'].str.contains('Bluetick Hound/'), 'breed'] = 'Bluetick Hound Mix'
working_df.loc[working_df['breed'].str.contains('Airedale Terrier/'), 'breed'] = 'Airedale Terrier Mix'
working_df.loc[working_df['breed'].str.contains('Finnish Spitz/'), 'breed'] = 'Finnish Spitz Mix'
working_df.loc[working_df['breed'].str.contains('Chesa Bay Retr/'), 'breed'] = 'Chesa Bay Retr Mix'
working_df.loc[working_df['breed'].str.contains('Schnauzer Giant/'), 'breed'] = 'Schnauzer Giant Mix'
working_df.loc[working_df['breed'].str.contains('Bernese Mountain Dog/'), 'breed'] = 'Bernese Mountain Dog Mix'
working_df.loc[working_df['breed'].str.contains('Australian Terrier/'), 'breed'] = 'Australian Terrier Mix'
working_df.loc[working_df['breed'].str.contains('Alaskan Malamute/'), 'breed'] = 'Alaskan Malamute Mix'
working_df.loc[working_df['breed'].str.contains('Parson Russell Terrier/'), 'breed'] = 'Parson Russell Terrier Mix'
working_df.loc[working_df['breed'].str.contains('Toy Fox Terrier/'), 'breed'] = 'Toy Fox Terrier Mix'
working_df.loc[working_df['breed'].str.contains('Irish Terrier/'), 'breed'] = 'Irish Terrier Mix'
working_df.loc[working_df['breed'].str.contains('American Eskimo/'), 'breed'] = 'American Eskimo Mix'
working_df.loc[working_df['breed'].str.contains('Bulldog/'), 'breed'] = 'Bulldog Mix'
working_df.loc[working_df['breed'].str.contains('Leonberger/'), 'breed'] = 'Leonberger Mix'
working_df.loc[working_df['breed'].str.contains('Swedish Vallhund/'), 'breed'] = 'Swedish Vallhund Mix'
working_df.loc[working_df['breed'].str.contains('Treeing Walker Coonhound/'), 'breed'] = 'Treeing Walker Coonhound Mix'
working_df.loc[working_df['breed'].str.contains('Glen Of Imaal/'), 'breed'] = 'Glen Of Imaal Mix'
working_df.loc[working_df['breed'].str.contains('Bloodhound/'), 'breed'] = 'Bloodhound Mix'
working_df.loc[working_df['breed'].str.contains('Bullmastiff/'), 'breed'] = 'Bullmastiff Mix'
working_df.loc[working_df['breed'].str.contains('Tibetan Terrier/'), 'breed'] = 'Tibetan Terrier Mix'
working_df.loc[working_df['breed'].str.contains('Welsh Terrier/'), 'breed'] = 'Welsh Terrier Mix'
working_df.loc[working_df['breed'].str.contains('Cavalier Span/'), 'breed'] = 'Cavalier Span Mix'
working_df.loc[working_df['breed'].str.contains('St. Bernard Smooth Coat/'), 'breed'] = 'St. Bernard Smooth Coat Mix'
working_df.loc[working_df['breed'].str.contains('Landseer/'), 'breed'] = 'Landseer Mix'
working_df.loc[working_df['breed'].str.contains('Dogo Argentino/'), 'breed'] = 'Dogo Argentino Mix'
working_df.loc[working_df['breed'].str.contains('Patterdale Terr/'), 'breed'] = 'Patterdale Terr Mix'
working_df.loc[working_df['breed'].str.contains('Field Spaniel/'), 'breed'] = 'Field Spaniel Mix'
working_df.loc[working_df['breed'].str.contains('Feist/'), 'breed'] = 'Feist Mix'
working_df.loc[working_df['breed'].str.contains('Beauceron/'), 'breed'] = 'Beauceron Mix'
working_df.loc[working_df['breed'].str.contains('Schipperke/'), 'breed'] = 'Schipperke Mix'
working_df.loc[working_df['breed'].str.contains('Dogue De Bordeaux/'), 'breed'] = 'Dogue De Bordeaux Mix'
working_df.loc[working_df['breed'].str.contains('Boykin Span/'), 'breed'] = 'Boykin Span Mix'
working_df.loc[working_df['breed'].str.contains('Standard Poodle/'), 'breed'] = 'Standard Poodle Mix'
working_df.loc[working_df['breed'].str.contains('Canaan Dog/'), 'breed'] = 'Canaan Dog Mix'
working_df.loc[working_df['breed'].str.contains('Silky Terrier/'), 'breed'] = 'Silky Terrier Mix'
working_df.loc[working_df['breed'].str.contains('Presa Canario/'), 'breed'] = 'Presa Canario Mix'
working_df.loc[working_df['breed'].str.contains('Treeing Cur/'), 'breed'] = 'Treeing Cur Mix'
working_df.loc[working_df['breed'].str.contains('Podengo Pequeno/'), 'breed'] = 'Podengo Pequeno Mix'
working_df.loc[working_df['breed'].str.contains('German Wirehaired Pointer/'), 'breed'] = 'German Wirehaired Pointer Mix'
working_df.loc[working_df['breed'].str.contains('Newfoundland/'), 'breed'] = 'Newfoundland Mix'
working_df.loc[working_df['breed'].str.contains('St. Bernard Rough Coat/'), 'breed'] = 'St. Bernard Rough Coat Mix'
working_df.loc[working_df['breed'].str.contains('Old English Bulldog/'), 'breed'] = 'Old English Bulldog Mix'
working_df.loc[working_df['breed'].str.contains('Boerboel/'), 'breed'] = 'Boerboel Mix'
working_df.loc[working_df['breed'].str.contains('Irish Wolfhound/'), 'breed'] = 'Irish Wolfhound Mix'
working_df.loc[working_df['breed'].str.contains('Keeshond/'), 'breed'] = 'Keeshond Mix'
working_df.loc[working_df['breed'].str.contains('Kuvasz/'), 'breed'] = 'Kuvasz Mix'
working_df.loc[working_df['breed'].str.contains('Cane Corso/'), 'breed'] = 'Cane Corso Mix'
working_df.loc[working_df['breed'].str.contains('American Foxhound/'), 'breed'] = 'American Foxhound Mix'
working_df.loc[working_df['breed'].str.contains('English Shepherd/'), 'breed'] = 'English Shepherd Mix'
working_df.loc[working_df['breed'].str.contains('English Foxhound/'), 'breed'] = 'English Foxhound Mix'
working_df.loc[working_df['breed'].str.contains('Chinese Crested/'), 'breed'] = 'Chinese Crested Mix'
working_df.loc[working_df['breed'].str.contains('Picardy Sheepdog/'), 'breed'] = 'Picardy Sheepdog Mix'
working_df.loc[working_df['breed'].str.contains('Skye Terrier/'), 'breed'] = 'Skye Terrier Mix'
working_df.loc[working_df['breed'].str.contains('Akbash/'), 'breed'] = 'Akbash Mix'
working_df.loc[working_df['breed'].str.contains('Wirehaired Pointing Griffon/'), 'breed'] = 'Wirehaired Pointing Griffon Mix'
working_df.loc[working_df['breed'].str.contains('Affenpinscher/'), 'breed'] = 'Affenpinscher Mix'
working_df.loc[working_df['breed'].str.contains('Mexican Hairless/'), 'breed'] = 'Mexican Hairless Mix'
working_df.loc[working_df['breed'].str.contains('Bearded Collie/'), 'breed'] = 'Bearded Collie Mix'
working_df.loc[working_df['breed'].str.contains('Greater Swiss Mountain Dog/'), 'breed'] = 'Greater Swiss Mountain Dog Mix'
working_df.loc[working_df['breed'].str.contains('French Bulldog/'), 'breed'] = 'French Bulldog Mix'
working_df.loc[working_df['breed'].str.contains('Nova Scotia Duck Tolling Retriever/'), 'breed'] = 'Nova Scotia Duck Tolling Retriever Mix'
working_df.loc[working_df['breed'].str.contains('Neapolitan Mastiff/'), 'breed'] = 'Neapolitan Mastiff Mix'
working_df.loc[working_df['breed'].str.contains('German Pinscher/'), 'breed'] = 'German Pinscher Mix'
working_df.loc[working_df['breed'].str.contains('English Cocker Spaniel/'), 'breed'] = 'English Cocker Spaniel Mix'
working_df.loc[working_df['breed'].str.contains('Treeing Tennesse Brindle/'), 'breed'] = 'Treeing Tennesse Brindle Mix'
working_df.loc[working_df['breed'].str.contains('Ibizan Hound/'), 'breed'] = 'Ibizan Hound Mix'
working_df.loc[working_df['breed'].str.contains('English Setter/'), 'breed'] = 'English Setter Mix'
working_df.loc[working_df['breed'].str.contains('Bull Terrier Miniature/'), 'breed'] = 'Bull Terrier Miniature Mix'
working_df.loc[working_df['breed'].str.contains('Welsh Springer Spaniel/'), 'breed'] = 'Welsh Springer Spaniel Mix'
working_df.loc[working_df['breed'].str.contains('Coton De Tulear/'), 'breed'] = 'Coton De Tulear Mix'
working_df.loc[working_df['breed'].str.contains('Brittany/'), 'breed'] = 'Brittany Mix'
working_df.loc[working_df['breed'].str.contains('Dandie Dinmont/'), 'breed'] = 'Dandie Dinmont Mix'
working_df.loc[working_df['breed'].str.contains('Belgian Sheepdog/'), 'breed'] = 'Belgian Sheepdog Mix'
working_df.loc[working_df['breed'].str.contains('Port Water Dog/'), 'breed'] = 'Port Water Dog Mix'
working_df.loc[working_df['breed'].str.contains('Bedlington Terr/'), 'breed'] = 'Bedlington Terr Mix'
working_df.loc[working_df['breed'].str.contains('Gordon Setter/'), 'breed'] = 'Gordon Setter Mix'
working_df.loc[working_df['breed'].str.contains('Eng Toy Spaniel/'), 'breed'] = 'Eng Toy Spaniel Mix'
working_df.loc[working_df['breed'].str.contains('Dachshund Stan/'), 'breed'] = 'Dachshund Stan Mix'
working_df.loc[working_df['breed'].str.contains('English Springer Spaniel/'), 'breed'] = 'English Springer Spaniel Mix'
working_df.loc[working_df['breed'].str.contains('Clumber Spaniel/'), 'breed'] = 'Clumber Spaniel Mix'
working_df.loc[working_df['breed'].str.contains('Saluki/'), 'breed'] = 'Saluki Mix'
working_df.loc[working_df['breed'].str.contains('Grand Basset Griffon Vendeen/'), 'breed'] = 'Grand Basset Griffon Vendeen Mix'
working_df.loc[working_df['breed'].str.contains('Hovawart/'), 'breed'] = 'Hovawart Mix'
working_df.loc[working_df['breed'].str.contains('Samoyed/'), 'breed'] = 'Samoyed Mix'
working_df.loc[working_df['breed'].str.contains('Jindo/'), 'breed'] = 'Jindo Mix'
working_df.loc[working_df['breed'].str.contains('Lakeland Terrier/'), 'breed'] = 'Lakeland Terrier Mix'
working_df.loc[working_df['breed'].str.contains('Lowchen/'), 'breed'] = 'Lowchen Mix'
working_df.loc[working_df['breed'].str.contains('Norwegian Elkhound/'), 'breed'] = 'Norwegian Elkhound Mix'
working_df.loc[working_df['breed'].str.contains('Otterhound/'), 'breed'] = 'Otterhound Mix'
working_df.loc[working_df['breed'].str.contains('Belgian Tervuren/'), 'breed'] = 'Belgian Tervuren Mix'
working_df.loc[working_df['breed'].str.contains('Sealyham Terr/'), 'breed'] = 'Sealyham Terr Mix'
working_df.loc[working_df['breed'].str.contains('Japanese Chin/'), 'breed'] = 'Japanese Chin Mix'
working_df.loc[working_df['breed'].str.contains('Afghan Hound/'), 'breed'] = 'Afghan Hound Mix'
working_df.loc[working_df['breed'].str.contains('Entlebucher/'), 'breed'] = 'Entlebucher Mix'

In [12]:
mix_df = working_df[working_df['breed'].str.contains('Mix')]\
        .sort_values(by=['animal_id'], ascending=False)
pure_df = working_df[~working_df['breed'].str.contains('Mix')]\
        .sort_values(by=['animal_id'], ascending=False)

Do not touch following two cells after running -- take a long time to complete

In [17]:
num_stay_df = pd.DataFrame(columns=["animal_id", "num_stays"])
for animal_id in working_df["animal_id"].unique():
    num_stay_df.loc[animal_id] = find_num_stays(working_df[working_df["animal_id"]==animal_id])

In [23]:
mean_stay_df = pd.DataFrame(columns=["animal_id", "avg_stay_len"]) 
for animal_id in working_df["animal_id"].unique():
    mean_stay_df.loc[animal_id] = find_staymean(working_df[working_df["animal_id"]==animal_id])

In [53]:
agg_stays_df = pd.merge(num_stay_df["num_stays"], mean_stay_df["avg_stay_len"], how="left", left_index=True, right_index=True)
agg_stays_df = agg_stays_df[agg_stays_df["num_stays"]!=0]
agg_stays_df["avg_stay_len"] = agg_stays_df["avg_stay_len"].apply(round_off)
agg_stays_df = agg_stays_df.reset_index().rename(columns={"index":"animal_id"})

In [46]:
working_df2 = working_df.loc[
    working_df["animal_id"]
    .isin(agg_stays_df["animal_id"].values)
]
agg_stays_df = agg_stays_df.set_index("animal_id", drop=True)

working_df2 = working_df2\
    .set_index("animal_id", drop=True)\
    .assign(num_stays = agg_stays_df["num_stays"])\
    .assign(avg_stay_len = agg_stays_df["avg_stay_len"])\
    .reset_index()

In [48]:
grouped_dogs_df = working_df2.groupby("animal_id").first()

In [50]:
dog_columns = list(grouped_dogs_df.columns)
for column in ["animal_type", "outcome_subtype", "is_intake", "date_of_birth"]:
    dog_columns.remove(column)
final_df = grouped_dogs_df[dog_columns].reset_index()
final_df["age"] = final_df["age"].apply(convert_age_to_num)

In [56]:
results_file = os.path.join("results_data", "Animal_Shelter_Final.csv")
final_df.to_csv(results_file, index=False)

In [ ]:
USER = "Edward"
PASSWORD = "RootinTootinSQLBootin7869!"
HOST = "127.0.0.1"
PORT = "3306"
DATABASE = "animal_shelter_db"
TABLENAME = "combined_intakes_and_outcomes"
# create the dataframe
# final_df=pd.read_csv("Animal_Shelter_Final.csv")
engine = create_engine(f"mysql+pymysql://{USER}:{PASSWORD}@{HOST}:{PORT}")
try:
    engine.execute(f"CREATE DATABASE {DATABASE}")
except ProgrammingError:
    warnings.warn(
        f"Could not create database {DATABASE}. Database {DATABASE} may already exist."
    )
    pass
engine.execute(f"USE {DATABASE}")
engine.execute(f"DROP TABLE IF EXISTS {TABLENAME}")
final_df.to_sql(name=TABLENAME, con=engine)